# Temporal filters



## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import numpy.fft as ufft

## Helper functions

In [ ]:
def getConj(v1, v2):
    return ufft.fftshift(np.append(v1[:-1], np.conj(np.flip(v2[1::]))))

def getAmpSpectrum(v):
    return np.abs(ufft.fftshift(ufft.fft(v)))

def getImpulseRes(v):
    return np.real(ufft.ifftshift(ufft.ifft(ufft.ifftshift(v))))

# Implementation of different temporal filters

In [ ]:
# Common parameters
Nt = 100                     # N time steps
dt = 0.005                   # step size in s
t = np.arange(0, Nt) * dt

## V1 (Zheng et al., 2007)

One drawback of using this temporal filter is that we manually set DC=0.
Therefore, there is a small unsmooth transition at low SFs.

Solution: Maybe rather use Watson-filter?

In [ ]:
def create_v1_tf(Nt=100, dt=0.001):
    m1 = 69.3; m2 = 22.9; m3 = 8.1; m4 = 0.8  # Zheng2007-params
    
    # Compute transfer function
    w = np.linspace(0, .5*(1/dt), int(Nt/2)+1); w[0] = 1. # prevent divison by 0
    H = m1 * np.exp(-(w / m2) ** 2.) / (1. + (m3 / w)**m4)
    
    # Add phase information
    x = np.linspace(0., .5*(1/dt), len(H))
    H = H * np.sin(x) + H * np.cos(x) * 1j
    w[0] = 0; w = getConj(w, -w); H[0] = H[0]*0; H = getConj(H, H)
    
    # Perform ifft to get impulse response
    h = np.real(ufft.ifft(ufft.ifftshift(H)))
    return h, np.abs(H), w

# Create filter and visualize
hz, Hz, wz = create_v1_tf(Nt, dt)

plt.figure(figsize=(12, 2))
plt.subplot(121); plt.plot(wz, Hz), plt.title("Frequency space")
plt.subplot(122); plt.plot(t,  hz), plt.title("Time space")
plt.show()

## Temporal CSF (Watson, 1986)

In [ ]:
def create_csf_tf(Nt=100, dt=0.001):
    # Watson1986-parameters to fit Robson1966 data. Zeta is .9 but then DC != 0
    kappa = 1.33; n1 = 9; n2 = 10; tau = 4.3/1000; zeta = 1; ksi = 269

    # Compute impulse response
    t = np.arange(0, Nt, 1) * dt
    h1 = (t / tau)**(n1 - 1) * np.exp(-t / tau) / (tau * math.factorial(n1-1))
    h2 = (t / (tau*kappa))**(n2 - 1) * np.exp(-t / (tau*kappa)) / (tau*kappa * math.factorial(n2-1))
    h = ksi * (h1 - zeta * h2)

    # Compute transfer function
    w = np.linspace(0, .5* (1/dt), int(Nt/2)+1); w2pij = w * 2. * np.pi * 1j
    H1 = (w2pij * tau + 1)**(-n1)
    H2 = (w2pij * tau*kappa + 1)**(-n2)
    H = np.abs(ksi * (H1 - zeta * H2))
    return h*dt, getConj(H, H), getConj(w, -w)

# Create filter and visualize
hw, Hw, ww = create_csf_tf(Nt, dt)

plt.figure(figsize=(12, 2))
plt.subplot(121); plt.plot(ww, Hw), plt.title("Frequency space")
plt.subplot(122); plt.plot(t,  hw), plt.title("Time space")
plt.show()

## M-cell filter (Benardete & Kaplan, 1999)

In [ ]:
def create_mcell_tf(Nt=100, dt=0.001):
    # Benardete-parameters. Units changed to seconds. Hs is .98 but then integral of impulse function != 0
    c = .4; A = 567; D = 2.2/1000.; C_12 = 0.056; T0 = 54.6; tau_L = 1.41/1000.; N_L = 30.3; Hs = 1.

    # Compute transfer function
    w = np.linspace(0, .5*(1./dt), int(Nt/2)+1); w2pij = w*2.*np.pi*1j
    H = A * np.exp(-w2pij*D) * (1. - Hs/(1. + w2pij*(T0/(1.+(c/C_12)**2.))/1000.)) * ((1./(1.+w2pij*tau_L))**N_L)
    H = getConj(H, H)

    # Perform ifft to get impulse response
    h = np.real(ufft.ifft(ufft.ifftshift(H)))
    return h, np.abs(H), getConj(w, -w)

# Create filter and visualize
hm, Hm, wm = create_mcell_tf(Nt, dt)

plt.figure(figsize=(12, 2))
plt.subplot(121), plt.plot(wm, Hm), plt.title("Frequency space")
plt.subplot(122), plt.plot(t,  hm), plt.title("Time space")
plt.show()

## Kelly-CSF (1979)

In [ ]:
def st_csf(sfs, tf):
    sfs = np.abs(sfs)                  # for negative sfs
    idx=np.where(sfs==0.); sfs[idx]=1. # fudge for sf=0
    idx2=np.where(tf==0.); tf[idx2]=1. # fudge for sf=0
    v = tf / sfs                       # calculate "velocity"
    
    # Calculate contrast sensitivity function:
    k = 6.1 + 7.3 * np.abs(np.log10(v/3.))**3.
    amax = 45.9 / (v + 2.)
    csf = k * v * (2.*np.pi*sfs)**2. * np.exp((-4.*np.pi*sfs) / amax)
    csfplt = 1. / csf
    
    if len(idx):
        csf[idx]=0.; csfplt[idx]= 0.; sfs[idx]=0.    # undo fudge
    if len(idx2):
        csf[idx2]=0.; csfplt[idx2]= 0.; tf[idx2]=0. # undo fudge
    return csf, csfplt

In [ ]:
tf = np.linspace(0, 100, 1000)
csf05, _ = st_csf(sfs=[.5,], tf=tf)
csf3, _ = st_csf(sfs=[3.,], tf=tf)
csf9, _ = st_csf(sfs=[9.,], tf=tf)

plt.figure(figsize=(8,3))
plt.subplot(121)
plt.plot(tf, csf05, label="0.5cpd"); plt.plot(tf, csf3, label="3cpd"); plt.plot(tf, csf9, label="9cpd");
plt.xlim(0.1,30); plt.yscale("log"); plt.xscale("log"); plt.ylim(0.5); plt.legend()
plt.subplot(122)
plt.plot(tf, csf05); plt.plot(tf, csf3); plt.plot(tf, csf9); plt.xlim(0,30)
plt.show()

# Visualize filters together

In [ ]:
plt.figure(figsize=(6, 2))
plt.subplot(121), plt.title("Frequency space")
plt.plot(ww, Hw/Hw.max(), label="watson-csf")
plt.plot(wz, Hz/Hz.max(), label="v1")
plt.plot(wm, Hm/Hm.max(), label="retina")
plt.plot(tf, csf05/csf05.max(), label="kelly-csf")
plt.legend(); plt.xlim(0, .5*(1/dt))

plt.subplot(122), plt.title("Time space")
plt.plot(t, hz/hz.max(), label="v1")
plt.plot(t, hw/hw.max(), label="csf")
plt.plot(t, hm/hm.max(), label="mcell"); plt.legend(); plt.xlim(0, Nt*dt);
#plt.savefig('temporal_filters.png', dpi=300)

# Sanity checks: Can we reproduce the filters in the opposite space?

## From time to frequency space

In [ ]:
plt.figure(figsize=(12, 2))
plt.subplot(131), plt.plot(wz, getAmpSpectrum(hz)), plt.plot(wz, Hz), plt.title('v1'), plt.xlim(0, .5*(1/dt))
plt.subplot(132), plt.plot(ww, getAmpSpectrum(hw)), plt.plot(ww, Hw), plt.title('csf'), plt.xlim(0, .5*(1/dt))
plt.subplot(133), plt.plot(wm, getAmpSpectrum(hm)), plt.plot(wm, Hm), plt.title('mcell'), plt.xlim(0, .5*(1/dt))
plt.show()

## From frequency to time space

Issue: we lost phase information.

In [ ]:
plt.figure(figsize=(12, 2))
plt.subplot(131), plt.plot(t, getImpulseRes(Hz)), plt.plot(t, hz), plt.title('v1')
plt.subplot(132), plt.plot(t, getImpulseRes(Hw)), plt.plot(t, hw), plt.title('csf')
plt.subplot(133), plt.plot(t, getImpulseRes(Hm)), plt.plot(t, hm), plt.title('mcell')
plt.show()

# Control: How many samples and which sampling frequency suffice?

In the active edge model paper, we used a sampling frequency of $f=100Hz$ and a time period of $T=0.2$ seconds.

In [ ]:
N2 = 40; dt2 = 0.005
h2, _, _ = create_csf_tf(N2, dt2)

N1 = 200; dt1 = 0.001
h1, _, _ = create_csf_tf(N1, dt1)

t1, t2 = np.arange(0,N1)*dt1, np.arange(0,N2)*dt2

plt.figure(figsize=(12, 2))
plt.subplot(121); plt.plot(t2, h2, '.-');          plt.plot(t1, h1),          plt.title("Unnormalized")
plt.subplot(122); plt.plot(t2, h2/h2.max(), '.-'); plt.plot(t1, h1/h1.max()), plt.title("Normalized")
plt.show()

print('Blue:\t f=%.0f,  T=%.2f' % (1/dt2, N2*dt2))
print('Orange:\t f=%.0f, T=%.2f' % (1/dt1, N1*dt1))